In [2]:
import sys
sys.path.append('..')
from config import RUTA_UNIDAD_ONE_DRIVE
from config import RUTA_LOCAL_ONE_DRIVE
from config import RUTA_COMPLETA

import pandas as pd
import geopandas as gpd

from dotenv import load_dotenv
import os

In [3]:
path_catastro = RUTA_UNIDAD_ONE_DRIVE + r'\OneDrive - Ingenio Azucarero Guabira S.A\INFORMACION GENERAL\CATASTRO\NUEVO\catastro_S13_SABADO.shp'

In [4]:
gdf_catastro = gpd.read_file(path_catastro)
gdf_cat_filtrado = gdf_catastro[
    (gdf_catastro['obs'] == 'ok') &
    (gdf_catastro['cultivo'].isin(['canha', 'siembra']))]
print('Area Total de Catastro:', gdf_cat_filtrado['area'].sum(), 'ha. para PROPIEDAD.xlsx')

Area Total de Catastro: 56139.12697181613 ha. para PROPIEDAD.xlsx


In [6]:
gdf_cat_filtrado.columns

Index(['fid', 'id', 'unidad_01', 'unidad_02', 'unidad_03', 'unidad_04',
       'unidad_05', 'variedad', 'soca', 'zona', 'textura', 'cultivo',
       'financia', 'fs', 'area', 'fc', 'obs', 'codco', 'nuevos', 'datosf',
       'mesc', 'edad', 'coducaf', 'geometry'],
      dtype='object')

In [8]:
invalidos = gdf_cat_filtrado[~gdf_cat_filtrado.is_valid]

In [16]:
invalidos[['fs']]

,fs
508,2023-06-27
10369,2022-05-11


In [22]:
import geopandas as gpd
import pandas as pd
import numpy as np

# Asegura geometrías válidas
gdf_cat_filtrado = gdf_cat_filtrado[gdf_cat_filtrado.is_valid]

# 🔄 Reproyectar a WGS84 (EPSG:4326)
gdf_cat_filtrado = gdf_cat_filtrado.to_crs(epsg=4326)

vertices_data = []

# Extraer vértices con atributos
for idx, row in gdf_cat_filtrado.iterrows():
    geometry = row.geometry
    if geometry.geom_type == 'Polygon':
        polys = [geometry]
    elif geometry.geom_type == 'MultiPolygon':
        polys = geometry.geoms
    else:
        continue

    for poly in polys:
        coords = list(poly.exterior.coords)
        for x, y in coords:
            data = row.drop('geometry').to_dict()
            data.update({'POINT_X': x, 'POINT_Y': y})
            vertices_data.append(data)

# Crear DataFrame
vertices_df = pd.DataFrame(vertices_data)

# Agregar columnas faltantes
vertices_df['OBJECTID'] = np.arange(1, len(vertices_df)+1)
vertices_df['Shape'] = 'Point ZM'
vertices_df['POINT_Z'] = 0
vertices_df['POINT_M'] = np.nan

# Convertir fs (fecha) a número Excel
vertices_df['fs'] = pd.to_datetime(vertices_df['fs'], errors='coerce')
excel_base = pd.Timestamp('1899-12-30')
vertices_df['fs'] = (vertices_df['fs'] - excel_base).dt.days.fillna(0).astype(int)

# Convertir columnas a enteros
cols_int = ['unidad_01', 'unidad_03', 'soca', 'zona']
vertices_df[cols_int] = vertices_df[cols_int].fillna(0).astype(int)

# Redondear coordenadas a 8 decimales
vertices_df['POINT_X'] = vertices_df['POINT_X'].round(8)
vertices_df['POINT_Y'] = vertices_df['POINT_Y'].round(8)

# Ordenar columnas como el Excel
columnas_finales = [
    'OBJECTID', 'Shape', 'id', 'unidad_01', 'unidad_02', 'unidad_03', 'unidad_04',
    'unidad_05', 'variedad', 'fs', 'textura', 'financia', 'soca', 'cultivo',
    'zona', 'area', 'POINT_X', 'POINT_Y', 'POINT_Z', 'POINT_M'
]

vertices_df = vertices_df[columnas_finales].sort_values(by='unidad_01').reset_index(drop=True)


In [23]:
vertices_df

,OBJECTID,Shape,id,unidad_01,unidad_02,unidad_03,unidad_04,unidad_05,variedad,fs,textura,financia,soca,cultivo,zona,area,POINT_X,POINT_Y,POINT_Z,POINT_M
0,107932,Point ZM,000010000L13.3,1,SONIMA,388,AGROPECUARIA MARIANA S.R.L.,L13.3,CITTCA0563,44659,F,IAG,2,canha,69,1.775559,-63.077887,-17.286836,0,NaN
1,107747,Point ZM,000010000L12,1,SONIMA,18358,ROJAS AMELUNGE CARLOS,L12,SP835073,45026,FY,IAG,1,canha,69,8.389535,-63.083055,-17.288103,0,NaN
2,107746,Point ZM,000010000L12,1,SONIMA,18358,ROJAS AMELUNGE CARLOS,L12,SP835073,45026,FY,IAG,1,canha,69,8.389535,-63.083647,-17.287868,0,NaN
3,107745,Point ZM,000010000L12,1,SONIMA,18358,ROJAS AMELUNGE CARLOS,L12,SP835073,45026,FY,IAG,1,canha,69,8.389535,-63.083805,-17.287818,0,NaN
4,107744,Point ZM,000010000L12,1,SONIMA,18358,ROJAS AMELUNGE CARLOS,L12,SP835073,45026,FY,IAG,1,canha,69,8.389535,-63.083836,-17.288163,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148826,148760,Point ZM,000022890000L1,2289,OKINAWA I 337--GUZMAN,42074,GUZMAN CARO JULIOMAR,L1,UCG9020,0,NaN,NaN,4,canha,304,48.264616,-62.862657,-17.262170,0,NaN
148827,148759,Point ZM,000022890000L1,2289,OKINAWA I 337--GUZMAN,42074,GUZMAN CARO JULIOMAR,L1,UCG9020,0,NaN,NaN,4,canha,304,48.264616,-62.862646,-17.262170,0,NaN
148828,148758,Point ZM,000022890000L1,2289,OKINAWA I 337--GUZMAN,42074,GUZMAN CARO JULIOMAR,L1,UCG9020,0,NaN,NaN,4,canha,304,48.264616,-62.862579,-17.262138,0,NaN
148829,148756,Point ZM,000022890000L1,2289,OKINAWA I 337--GUZMAN,42074,GUZMAN CARO JULIOMAR,L1,UCG9020,0,NaN,NaN,4,canha,304,48.264616,-62.862508,-17.261855,0,NaN
